# Calculation of AUROCs from Single-sample GSEA projection (ssGSEA) on TCGA Data 

## Background

Traditional gene set enrichment analysis (GSEA)<sup>5</sup> assesses the differential coordinate up- or down-regulation of a biological process or pathway between groups of samples belonging to two phenotypes. However, this method requires samples to be normalized for between sample comparisons, and does not allow the post-hoc assignment of a new sample to a phenotype group. In contrast, single-sample GSEA (ssGSEA)<sup>1</sup> allows the ability to assess the enrichment of gene sets in individual samples independently, but does not natively perform cross-phenotype comparisons. Through the use of receiver operating characteristic (ROC)<sup>4</sup> analysis, <b>this notebook performs cross-phenotype comparisons on ssGSEA quantifications to determine which gene sets are significantly differentially enriched between two phenotype groups</b>, and determines threshold values by which additional samples with unknown phenotypes can be assigned to one of the known groups. This method was first used to characterize the relationship between high expression of MTOR and the expression of a predictive drug response signature in Glioblastoma samples<sup>2</sup>.

First, samples are selected from TCGA based on expression levels of a marker gene (High expression and Low Expression). Second, ssGSEA is used to project each single sample’s full gene expression profile into the space of gene sets. It does this via *enrichment scores*, which represent the degree to which the genes in each particular gene set are coordinately up- or down- regulated within that sample.

Finally, the enrichment scores and the binary phenotype of High vs. Low expression of the selected marker gene are used to calculate the receiver operating characteristic (ROC) for each gene set, the area under the ROC curve (AUC), and the Matthews correlation coefficient (MCC)<sup>3</sup>. These metrics describe how well enrichment of a given gene set tracks with the defined phenotype (e.g. if a given signaling pathway is significantly differentially enriched in samples highly expressing vs. samples lowly expressing a gene of interest).

Since ssGSEA scores are determined on a per-sample basis, and each sample is individually normalized (for example, by transcripts per million), once a gene set is determined to be a good classifier based on the AUC and the MCC, new samples can be assigned to groups post-hoc based their ssGSEA enrichment score using a cutoff threshold (Youden's J statistic) if their true phenotype is unknown.

Note that while this notebook retrieves expression data from TCGA for use with ssGSEA_ROC, the ssGSEA_ROC method can be used with any ssGSEA compatible dataset starting from the "Project gene expression dataset..." section of the Stepwise Analysis with Descriptions section.

## Before you begin

You must log in to a GenePattern server. In this notebook we will use **```GenePattern Cloud```**

Note: if you are not familiar with GenePattern Notebook features, you can review them here: <a href="https://notebook.genepattern.org/services/sharing/notebooks/361/preview/">GenePattern Notebook Tutorial</a>.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Sign in to GenePattern by entering your username and password into the form below.
</div>

In [8]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

# One Step Analysis
Select this option to run the entire analysis pipeline automatically in one step. This requires minimal user intervention and is recomended for users already familiar with the principles of ssGSEA and ROC analysis and who don't need to modify ssGSEA's default parameters.

In [9]:
tcga_ssgsea_roc_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:689')
tcga_ssgsea_roc_job_spec = tcga_ssgsea_roc_task.make_job_spec()
tcga_ssgsea_roc_job_spec.set_parameter("TCGA.SampleSelection1.TCGA.Collection", "GBM")
tcga_ssgsea_roc_job_spec.set_parameter("TCGA.SampleSelection1.Gene.Symbol", "MTOR")
tcga_ssgsea_roc_job_spec.set_parameter("TCGA.SampleSelection1.High.Expression", "1")
tcga_ssgsea_roc_job_spec.set_parameter("TCGA.SampleSelection1.Low.Expression", "-1")
tcga_ssgsea_roc_job_spec.set_parameter("TCGA.SampleSelection1.MSigDB.Version", "latest")
tcga_ssgsea_roc_job_spec.set_parameter("ssGSEA2.gene.sets.database.files", "")
tcga_ssgsea_roc_job_spec.set_parameter("ssGSEA_ROC3.Reverse", "FALSE")
tcga_ssgsea_roc_job_spec.set_parameter("ssGSEA_ROC3.Plot.Top.Results", "20")
tcga_ssgsea_roc_job_spec.set_parameter("job.memory", "2 Gb")
tcga_ssgsea_roc_job_spec.set_parameter("job.queue", "gp-cloud-default")
tcga_ssgsea_roc_job_spec.set_parameter("job.cpuCount", "1")
tcga_ssgsea_roc_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(tcga_ssgsea_roc_task)

GPTaskWidget(lsid='urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:689'…

# Stepwise Analysis with Descriptions
Select this option to run each step of the analysis pipeline separately. This is ideal for first time users, users who would like more detailed information about each step of the analysis, or users who wish to exercise more fine-grained control over step parameters.

## Select TCGA Samples Based on Expression Level Criteria

We will use the GenePattern TCGA.SampleSelection module to retrieve TCGA datasets, then automatically select samples from within that dataset depending on their expression level of a specific gene of interest. This data will be returned as a .GCT file containing the selected samples, and a .CLS file containing the Sample to Gene Level (High expression vs. Low expression) assignments.

In [10]:
tcga_sampleselection_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:genepattern.org:module.analysis:00417')
tcga_sampleselection_job_spec = tcga_sampleselection_task.make_job_spec()
tcga_sampleselection_job_spec.set_parameter("TCGA.Collection", "GBM")
tcga_sampleselection_job_spec.set_parameter("Gene.Symbol", "MTOR")
tcga_sampleselection_job_spec.set_parameter("High.Expression", "1")
tcga_sampleselection_job_spec.set_parameter("Low.Expression", "-1")
tcga_sampleselection_job_spec.set_parameter("Output.Type", "scaled_estimate")
tcga_sampleselection_job_spec.set_parameter("MSigDB.Version", "7.4")
tcga_sampleselection_job_spec.set_parameter("job.memory", "4 Gb")
tcga_sampleselection_job_spec.set_parameter("job.queue", "gp-new-beta")
tcga_sampleselection_job_spec.set_parameter("job.cpuCount", "1")
tcga_sampleselection_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(tcga_sampleselection_task)

GPTaskWidget(lsid='urn:lsid:genepattern.org:module.analysis:00417')

## Project gene expression dataset into the space of selected gene sets

We will use the GenePattern ssGSEA analysis module to transform a gene expression dataset into a dataset where each row corresponds to a pathway, and each column is a sample. Each value in the new dataset will therefore represent the up- or downregulation of a pathway (row) within a sample (column).

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>

Provide the required parameters for the ssGSEA module below.

- For the **input gct file*** parameter, click on the dropdown and select output of the TCGA.SampleSelection module (it should end with `.gct`).
- For a detailed description of the parameters you can read the <a href='https://gsea-msigdb.github.io/ssGSEA-gpmodule/v10/index.html'>parameter documentation</a>.
- For a description of the <strong><em>gene sets database files*</em></strong> parameter options, visit <a href="https://www.gsea-msigdb.org/gsea/msigdb/index.jsp">the MSigDB webpage</a>.
- Click <strong><em>Run</em></strong> on the analysis below.</li>
</div>

In [11]:
ssgsea_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00270')
ssgsea_job_spec = ssgsea_task.make_job_spec()
ssgsea_job_spec.set_parameter("input.gct.file", "")
ssgsea_job_spec.set_parameter("output.file.prefix", "")
ssgsea_job_spec.set_parameter("gene.sets.database.files", "")
ssgsea_job_spec.set_parameter("gene.symbol.column", "Name")
ssgsea_job_spec.set_parameter("gene.set.selection", "ALL")
ssgsea_job_spec.set_parameter("sample.normalization.method", "none")
ssgsea_job_spec.set_parameter("weighting.exponent", "0.75")
ssgsea_job_spec.set_parameter("min.gene.set.size", "10")
ssgsea_job_spec.set_parameter("combine.mode", "combine.off")
ssgsea_job_spec.set_parameter("job.memory", "2 Gb")
ssgsea_job_spec.set_parameter("job.queue", "gp-new-beta")
ssgsea_job_spec.set_parameter("job.cpuCount", "1")
ssgsea_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(ssgsea_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00270')

## Visualize projected pathways as a heat map

We will use the GenePattern HeatMapViewer module to visualize the resulting projection of genes into pathways.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
- In the **dataset** parameter below, click on the dropdown and select output of the ssGSEA module (it should end with `.PROJ.gct`).
- Click **Run**.
</div>

In [12]:
heatmapviewer_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00010')
heatmapviewer_job_spec = heatmapviewer_task.make_job_spec()
heatmapviewer_job_spec.set_parameter("dataset", "")
genepattern.display(heatmapviewer_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00010')

## Calculate ROCs for each gene set from the projections and assigned phenotype

We will use the GenePattern ssGSEA_ROC module, which implements the R package ROCR<sup>4</sup> to analyze the coherence between an assigned phenotype for the samples (e.g. high or low expression of a specific marker gene) and enrichment of the selected gene sets. For each gene set, the Matthews correlation coefficient, the area under the curve (AUC), a two-sided wilcox test on the ssSGEA scores and a number of other metrics are calculated. The results are returned as a full (.txt) data table showing the scores and statistics for all gene sets, as well as a PDF plotting the top scored gene sets.

For datasets with ≥7 samples in both phenotypes, ssGSEA_ROC utilizes a modified version of the GSEA phenotype permutation algorithm to calcualte GSEA style nominal (NOM) pValues and FDRs on the basis of permutation of the sample-to-phenotype assignments. 

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
- For the <b>PROJ gct</b> (ssGSEA Projection Matrix) parameter, click the "Add File or URL..." box and select output of the ssGSEA module (it should end with `.PROJ.gct`).

- For the <b>CLS</b> (two phenotype CLS) file parameter, click the "Add File or URL..." box and select the `.CLS` file produced by TCGA.SampleSelection, or select the "Upload File..." button to provide the .CLS file with the phenotypic information (<a href="http://software.broadinstitute.org/cancer/software/genepattern/file-formats-guide#CLS">read about CLS files here</a>).

In [13]:
ssgsea_roc_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:177')
ssgsea_roc_job_spec = ssgsea_roc_task.make_job_spec()
ssgsea_roc_job_spec.set_parameter("PROJ.gct", "")
ssgsea_roc_job_spec.set_parameter("CLS", "")
ssgsea_roc_job_spec.set_parameter("Reverse", "FALSE")
ssgsea_roc_job_spec.set_parameter("Plot.Top.Results", "20")
ssgsea_roc_job_spec.set_parameter("job.memory", "2 Gb")
ssgsea_roc_job_spec.set_parameter("job.queue", "gp-new-beta")
ssgsea_roc_job_spec.set_parameter("job.cpuCount", "1")
ssgsea_roc_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(ssgsea_roc_task)

GPTaskWidget(lsid='urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:177'…

In [7]:
import pandas as pd
from io import BytesIO
import os
#import qgrid # qgrid is not installed in this kernel it may be in the python3.9

@genepattern.build_ui(parameters={
    "txt": {
        "name": "File to load",
        "description": "The TXT file to load.",
        "type": "file",
        "kinds": [".Results.txt"]
    },
    "how_many": {
        "name": "Rows to display",
        "description": "How many rows to display",
        "type": "int",
        "default": 5
    },
    "output_var": {
        "hide": True,
        "default": "input_data",
        "name": "input_data"
    }
}, name="View ssGSEA_ROC Results Table")
def read_txt(txt,how_many):
    print(txt)
    r = gp.GPFile(genepattern.session.sessions[0], txt)
    df = pd.read_table(BytesIO(r.open().read()),index_col=0)
    display(df.head(how_many))
    # qgrid is not installed in this kernel, if it were, you'd use:
    #display(qgrid.show_grid(df,precision=2,grid_options={'editable': False,'maxVisibleRows': how_many}))
    

UIBuilder(function_import='nbtools.tool(id="View ssGSEA_ROC Results Table", origin="Notebook").function_or_met…

## Interpreting Results

The ssGSEA_ROC file produces two outputs, a PDF and a TSV. 
- The PDF contains the plotted ROC curves with cutoff values, corresponding boxplots of the ssGSEA scores, and Wilcox pValues for the number of sets selected by the Plot Top Results parameter.
- The TSV file is a tab delimited text file that contains the complete statistics for all gene sets that were included in the `.PROJ.GCT`.

The gene sets enriched in the first phenotype (or second phenotype if Reverse=TRUE) will exhibit positive MCCs, and AUCs >0.5.
The gene sets enriched in the second phenotype (or first phenotype if Reverse=TRUE) will exhibit negative MCCs, and AUCs <0.5.

# References

1. Barbie DA, Tamayo P, et al. Systematic RNA interference reveals that oncogenic KRAS-driven cancers require TBK1. Nature. 2009;462:108-112. https://pubmed.ncbi.nlm.nih.gov/19847166/

2. Benitez JA, Finlay D, Castanza A, Parisian AD, Ma J, Longobardi C, Campos A, Vadla R, Izurieta A, Scerra G, Koga T, Long T, Chavez L, Mesirov JP, Vuori K, Furnari F. PTEN deficiency leads to proteasome addiction: a novel vulnerability in glioblastoma. Neuro Oncol. 2021 Jul 1;23(7):1072-1086. doi: 10.1093/neuonc/noab001. PMID: 33428749.

3. Chicco, D., Jurman, G. The advantages of the Matthews correlation coefficient (MCC) over F1 score and accuracy in binary classification evaluation. BMC Genomics 2020;21:6. https://doi.org/10.1186/s12864-019-6413-7

4. Sing T, Sander O, Beerenwinkel N, Lengauer T. ROCR: visualizing classifier performance in R. Bioinformatics. 2005;21(20):3940-3941. https://doi.org/10.1093/bioinformatics/bti623

5. Subramanian A, Tamayo P, Mootha VK, Mukherjee S, Ebert BL, Gillette MA, Paulovich A, Pomeroy SL, Golub TR, Lander ES, Mesirov JP. Gene set enrichment analysis: A knowledge-based approach for interpreting genome-wide expression profiles. PNAS. 2005;102(43):15545-15550. http://www.pnas.org/content/102/43/15545.abstract


- MSigDB website (http://www.msigdb.org)
- GSEA website (https://www.gsea-msigdb.org)
